In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import pandas as pd
import os

In [2]:
class SkipGramModel(nn.Module):
    """
    Skip gram model of word2vec
    """
    def __init__(self, vocab_size, embed_dim):
        """
        Init parameter
        vocab_size: word numberv of vocabulary
        embed_dim: Embedding dimension, typically from 50 to 500
        """
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        # input embedding project input word to a vector
        self.input_embed = nn.Embedding(vocab_size, embed_dim, sparse=True)
        # output embedding project output word to a vector
        self.output_embed = nn.Embedding(vocab_size, embed_dim, sparse=True)
        
    def init_embed(self):
        """
        Init embedding weight
        input_embed is a uniform distribution is [-0.5/vocab_size, 0.5/vocab_size]
        """
        init_range = 0.5/self.vocab_size
        self.input_embed.weight.data.uniform_(-init_range, init_range)
        self.output_embed.weight.data.uniform_(-0,0)
        
    def forward(self, pos_center_word, pos_neighbor_word, neg_neighbor_word):
        """
        pos_center_word: list of center word id [1,3,5,...]
        pos_neighbor_word: list of neighbor word id [3,5,6,...]
        neg_neighbor_word: list[list] of negative neigbor word by negative sample the inner list is every
        pos_neighbor_word sample n negative neighbor word [[2,4,5,7,8],[1,2,3,4,6],[1,2,3,4,5],...]
        """
        # input word(center word) embeding
        pos_center_embed = self.input_embed(pos_center_word)
        # caculate positive sample loss
        pos_neighbor_embed = self.output_embed(pos_neighbor_word)
        pos_score = torch.mul(pos_center_embed, pos_neighbor_embed)
        pos_score = torch.sum(pos_score, dim=-1)
        pos_score = F.logsigmoid(pos_score)
        
        # caculate negative sample loss
        # neg_neighbor_embed is a three dimension matrix different to pos_neighbor_embed
        neg_neighbor_embed = self.output_embed(neg_neighbor_word) 
        neg_score = torch.bmm(neg_neighbor_embed, pos_center_embed.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1*neg_score)
        
        # loss
        loss = -1*(torch.sum(pos_score) + torch.sum(neg_score))
        return loss

In [7]:
class Word2Vec:
    """
    Word2vec model
    """
    def __init__(self, data,batch_size=128 ,iters=10,learning_rate=0.01,embeding_dim=100,windows_size=5, min_count=5):
        """
        data: list[list] train data word id list for sentence
        embeding_dim: embedding dimension
        windows size: int, windows number for search neighbor word of a given center word
        min_count: int, filter word if word frequency less than min_count 
        """
        self.data = data
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embeding_dim = embeding_dim
        self.windows_size = windows_size
        self.min_count = min_count
        self.iters = iters
        self.word2id = dict()
        self.id2word = dict()
        self.wordfrequency = self.dataprocess(data)
        self.vocab_size = len(self.wordfrequency)
        self.sample_table = self.init_sample_table()
        self.model = SkipGramModel(self.vocab_size+1, self.embeding_dim)
        self.optim = optim.SGD(self.model.parameters(), self.learning_rate)
    
    def dataprocess(self, data):
        """
        filter word frequency less than min count and than construct a new word frequency dict
        note:
            the new word frequency dict is use the new word2id index project and we must save the
            word2id and id2word project dict
        """
        # count the data word frequency
        wordfrequency = dict()
        # filter word and use the new index to construct the dict
        word_frequency = dict()
        
        for sentence in data:
            for word in sentence:
                if word not in wordfrequency:
                    wordfrequency[word] = 1
                else:
                    wordfrequency[word] += 1
                    
        idx = 0
        for word, val in wordfrequency.items():
            if val < self.min_count:
                continue
            self.word2id[word] = val
            self.id2word[idx] = word
            word_frequency[idx] = val
            idx += 1
        return word_frequency
    
    def init_sample_table(self):
        """
        init teh sample tabel for negative sample 
        note:
            sample_table_size is 10e8 like the word2vec doc and sample ratio is also use the offical paper
        """
        sample_table_size = 10e4
        sample_table = []
        pow_frequency = np.array(list(self.wordfrequency.values()))**0.75
        ratio = pow_frequency/sum(pow_frequency)
        count = np.round(ratio*sample_table_size)
        for i, c in enumerate(count):
            sample_table += [i] * int(c)
        sample_tabel = np.array(sample_table)
        return sample_table
        
    def generate_train_data(self):
        """
        generate the train data like (center word, neighbor word) pairs
        """
        train_data = []
        for sentence in self.data:
            for i, word_u in enumerate(sentence):
                for j, word_v in enumerate(sentence[max(i-self.windows_size,0):i+self.windows_size]):
                    if i == j:
                        continue
                    train_data.append((word_u, word_v))
        return train_data
    
    def negative_sample_batch(self):
        """
        the negative sample without ignore the target word(may be the negative sample word is the same
        to the neighbor word)
        """
        return np.random.choice(self.sample_table, size=(self.batch_size, 5)).tolist()
    
    def train_batch(self, train_data):
        """
        a generator for generate the train batch like [(center_word, neighbor_word),(center_word, neighbor_word)]
        """
        iters = len(train_data)//self.batch_size + 1
        for i in range(iters-1):
            start = i*self.batch_size
            end = (i+1)*self.batch_size
            yield train_data[start:end]
    
    def train(self):
        for k in range(self.iters):
            train_data = self.generate_train_data()
            dataloader = self.train_batch(train_data)
            for i, data in enumerate(dataloader):
                pos_centor_word = Variable(torch.LongTensor([pair[0] for pair in data]))
                pos_neighbor_word = Variable(torch.LongTensor([pair[1] for pair in data]))
                neg_neighbor_word = Variable(torch.LongTensor(self.negative_sample_batch()))
                self.optim.zero_grad()
                loss = self.model(pos_centor_word, pos_neighbor_word, neg_neighbor_word)
                loss.backward()
                self.optim.step()
            print("Epoch %d is finished" %(k+1))

In [8]:
# 测试数据
data = np.random.randint(1,50,size=(1000,8)).tolist()

In [9]:
word2vec = Word2Vec(data)

In [10]:
%%time
word2vec.train()

Epoch 1 is finished
Epoch 2 is finished
Epoch 3 is finished
Epoch 4 is finished
Epoch 5 is finished
Epoch 6 is finished
Epoch 7 is finished
Epoch 8 is finished
Epoch 9 is finished
Epoch 10 is finished
CPU times: user 1min 17s, sys: 1.54 s, total: 1min 18s
Wall time: 28.7 s


In [11]:
# the vecotr of every word after training 
word2vec.model.input_embed.weight

Parameter containing:
tensor([[-2.4881e-02,  8.8162e-01,  6.3242e-02,  ...,  1.2183e+00,
          3.6513e-01, -1.3111e+00],
        [-3.6192e-01, -1.1458e-01, -6.4307e-02,  ...,  6.8155e-01,
          2.1675e-02,  1.3356e-01],
        [ 5.7432e-01,  5.0174e-01, -3.0563e-01,  ..., -1.7531e-01,
          1.1257e+00, -1.0213e-01],
        ...,
        [ 3.6956e-01,  6.7997e-01, -3.2813e-01,  ..., -5.2111e-01,
          1.1278e-01, -1.3773e-02],
        [ 1.0463e-03, -2.5979e-01, -3.9221e-02,  ...,  1.4790e-02,
         -5.0223e-02,  2.0891e-01],
        [-3.9398e-01,  7.6720e-01,  3.1329e-01,  ...,  3.8423e-01,
          4.4567e-01,  4.7976e-01]], requires_grad=True)